In [1]:
import os
import yaml
import pandas as pd
import numpy as np
import multiprocessing as mp
from functools import partial
import h3
from tsnecuda import TSNE
import math
import json
import gc
from collections import OrderedDict
##For merging sets of dictionaries
from mergedeep import merge as dict_merge
from mergedeep import  Strategy
import sys
import copy
import pickle
import multiprocessing as mp
from datetime import date
from dateutil.relativedelta import relativedelta

from tqdm import tqdm
import itertools
from collections import defaultdict
import random

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from datetime import date
import re

import geohash as gh

# import sys
# sys.path.insert(1, '../hve_challenger_train_sales_pipeline/hve_challenger_train_sale_dp/local_dependencies/')
# from geoTree.utils import percentiles_list
# from geoTree.geoTree import GeoTree

##Modeling Related Packages
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch.autograd import Variable

import gpytorch
import pytorch_forecasting
from pytorch_forecasting.models.temporal_fusion_transformer.sub_modules import GatedResidualNetwork, VariableSelectionNetwork, GatedLinearUnit, AddNorm ,GateAddNorm

from sklearn.preprocessing import RobustScaler, StandardScaler , OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight ,compute_sample_weight
from sklearn.inspection import PartialDependenceDisplay
from sklearn.base import BaseEstimator, TransformerMixin, _OneToOneFeatureMixin 
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion 
from sklearn.utils.validation import check_is_fitted
#from sklearn.manifold import TSNE
from MulticoreTSNE import MulticoreTSNE as TSNE
##Interpretability
import shap as shp
#from tsnecuda import TSNE

##Plotting Libs
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
##Path to local dependencies
#from torch_local.custom_layers import GMLPBlock, SegmentedMLP, self_attention
#from torch_local.loss_functions import loss_pc10_adaptive

##Model Tunning
import ray
from ray import tune,air
from ray.tune import JupyterNotebookReporter
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.search.bohb import TuneBOHB
from ray.tune.schedulers import ASHAScheduler
import pytorch_lightning as pl
from pytorch_lightning import trainer
from pytorch_lightning.loggers import TensorBoardLogger
from ray.tune.integration.pytorch_lightning import TuneReportCallback, TuneReportCheckpointCallback
from ray.air import session
from pytorch_lightning.callbacks import ModelCheckpoint
from ray.air.config import RunConfig, ScalingConfig, CheckpointConfig
from ray.train.lightning import (
    LightningTrainer,
    LightningConfigBuilder,
    LightningCheckpoint,
)
import pickle
import dill
from scipy.ndimage import gaussian_filter1d
from scipy.signal.windows import triang
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, pipeline,AutoModelForCausalLM

import warnings
warnings.filterwarnings("ignore", message=".*The 'nopython' keyword.*")

/home/jovyan/.local/lib/python3.8/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/home/jovyan/.local/lib/python3.8/site-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def delta_minimization_order(all_masks, max_swa

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("-----device:{}".format(device))
print("-----Pytorch version:{}".format(torch.__version__))
print(f"gpu available: {torch.cuda.is_available()}")

-----device:cpu
-----Pytorch version:1.12.1+cu102
gpu available: False


In [4]:
percentiles_list = [.01,.05,.1,.25,.5,.75,.9,.95,.99]

In [5]:
##Set some global Vars and functions

def pc10_calc(y, y_pred):    
    return round(np.mean((np.abs(y-y_pred)/y)<=0.1),3)


def pc10_accm(y, y_pred):    
    return np.sum((np.abs(y-y_pred)/y)<=0.1)


def pc10plus_accm_torch(y,y_pred):
    ratio = y_pred/y
    return torch.sum(ratio > 1.1)


def pc10_accm_torch(y,y_pred):
    diff = torch.abs(y-y_pred)/y
    return torch.sum(diff<= 0.1)

def median_bias(y, y_pred):
    return round(((y_pred - y) / y).median(), 3)

def pc10plus (y, y_pred):
    return round(np.mean((y_pred/y) > 1.1), 3)

def mape(y,y_pred):
    diff = np.abs((y-y_pred)/y)
    return round(np.mean(diff),3)

# for dataframe
def df_metric_wArg(df,colTrue,colPred):
    return pd.Series({'count'      : df.shape[0],
                      'pc10'      : pc10_calc(df.loc[:, colTrue], df.loc[:, colPred]),
                      'pc10+'     : pc10plus(df.loc[:, colTrue], df.loc[:, colPred]),
                      'median_bias': median_bias(df.loc[:, colTrue], df.loc[:, colPred]),
                      'mape' : mape(df.loc[:, colTrue], df.loc[:, colPred])
                     })

def adj_col_list(list_to_adj,ref_list,retain=True):
    ##If True keep elements in ref_list
    if retain:
        result = [col for col in list_to_adj if col in ref_list]
    else:
        result = [col for col in list_to_adj if col not in ref_list]
    return result

In [6]:
#Set the State
state = ['WA']
trgt_cnty = ['WA063']
time = '2023m05'
ple_size = 45

In [7]:
datapath = f'/home/jovyan/challenger/data'
artpath = f'/home/jovyan/challenger/artifacts'
print(datapath)
print(artpath)

/home/jovyan/challenger/data
/home/jovyan/challenger/artifacts


In [8]:
%%time
bluesky_data_train = pd.read_pickle(f'{datapath}/raw/bluesky_data_train_state_full_{time}_extended.pkl')
bluesky_data_train=bluesky_data_train[bluesky_data_train['final_state'].isin(state)].reset_index(drop=True)

CPU times: user 46.8 s, sys: 24.1 s, total: 1min 10s
Wall time: 1min 10s


In [9]:
bluesky_data_test =  pd.read_pickle(f'{datapath}/raw/bluesky_data_test_state_full_{time}_extended.pkl')
bluesky_data_test=bluesky_data_test[bluesky_data_test['final_state'].isin(state)].reset_index(drop=True)

In [10]:
%%time
##If data already created then load from disk 
#Load the preprocessor
print('**Loading Preprocessor**')
with open(f'{artpath}/{state[0]}_{time}_ple{ple_size}_processor_new_h3_txt_extended.dill', 'rb') as dill_file:
    preprocessor=dill.load(dill_file)
print('**Loading Preprocessor-Done**')

#Load Preprocessed Train Data
print('**Loading Training Preprocessed Data**')
X_train_s= pd.read_pickle(f'{datapath}/preprocessed/{state[0]}_{time}_ple{ple_size}_X_train_s_new_h3_txt_extended.pkl').reset_index(drop=True)
print('**Loading Training Preprocessed Data-Done**')

#Load Preprocessed Test Data
print('**Loading Test Preprocessed Data**')
X_test_s= pd.read_pickle(f'{datapath}/preprocessed/{state[0]}_{time}_ple{ple_size}_X_test_s_new_state_h3_txt_extended.pkl').reset_index(drop=True)
print('**Loading Test Preprocessed Data-Done**')

#Load the input dict 
print('**Loading Input Dict**')
with open(f'{artpath}/{state[0]}_{time}_ple{ple_size}_input_dict_new_h3_txt_extended.dill', 'rb') as dill_file:
    input_dict=dill.load(dill_file)
print('**Loading Input Dict-Done**')

**Loading Preprocessor**
**Loading Preprocessor-Done**
**Loading Training Preprocessed Data**
**Loading Training Preprocessed Data-Done**
**Loading Test Preprocessed Data**
**Loading Test Preprocessed Data-Done**
**Loading Input Dict**
**Loading Input Dict-Done**
CPU times: user 55.9 s, sys: 5.28 s, total: 1min 1s
Wall time: 1min 1s


In [11]:
X_test_s.shape

(8800, 153)

In [12]:
X_test_s=X_test_s[['ccmls_cnt_bdrm', 'ccmls_cnt_btrm', 'ccmls_cnt_ppty_stry', 'ccmls_cnt_unit', 'ccmls_dt_blt_yr', 'ccmls_nbr_gla', 'clmls_cnt_bdrm', 'clmls_cnt_tot_btrm', 'clmls_cnt_bldng_stry_cl', 'clmls_cnt_unit', 'clmls_cnt_grge_car', 'clmls_dt_blt_yr', 'clmls_nbr_area_livng', 'clmls_cnt_prkng_spce_open','clmls_cnt_prkng_spce_tot', 'fre_cnt_unit', 'fatax_amt_tot_asesd_val', 'fatax_amt_asesd_imprmt_val', 'fatax_amt_asesd_lnd_val', 'fatax_cnt_bdrm', 'fatax_cnt_bldng_stry', 'fatax_cnt_prkng_spce', 'fatax_cnt_tot_btrm_hve', 'fatax_cnt_unit', 'fatax_dt_eff_blt_yr', 'fatax_nbr_livng_sq_ft', 'fatax_nbr_lot_size_sq_ft', 'fatax_amt_tot_mkt_val', 'fatax_dt_cnsttn_yr', 'fatax_dt_tax_yr', 'idmtax_amt_asesd_tot_val', 'idmtax_amt_asesd_imprmt_val', 'idmtax_amt_asesd_lnd_val', 'idmtax_cnt_bdrm', 'idmtax_cnt_bldng_stry_drvd', 'idmtax_cnt_grge_car', 'idmtax_cnt_tot_btrm_hve', 'idmtax_cnt_unit', 'idmtax_dt_cnsttn_yr', 'idmtax_nbr_lot_size_sq_ft', 'idmtax_nbr_unvrsl_bldng_sq_ft_drvd', 'idmtax_amt_tot_mkt_val', 'idmtax_dt_tax_yr', 'lat', 'long', 'ucdp_cnt_prkng_spce', 'ucdp_cnt_tot_btrm_hve', 'ucdp_cnt_bdrm', 'ucdp_cnt_unit_hve', 'ucdp_effectiveage', 'ucdp_nbr_bldng_sq_ft_hve', 'ucdp_num_grge_hve', 'ucdp_quality_of_construction', 'ucdp_site_sqft', 'ucdp_stories', 'ucdp_year_built', 'ucdp_salescompsubjactualage', 'mortgage_rate_lag1', 'mortgage_rate_lag2', 'mortgage_rate_lag3', 'mortgage_rate_lag4', 'mortgage_rate_lag5', 'mortgage_rate_lag6', 'avg_price_3m_bdrm_geohash6_orig','avg_price_6m_bdrm_geohash6_orig','avg_price_12m_bdrm_geohash6_orig', 'purprice_lag1', 'csushpinsa_lag1', 'csushpinsa_lag2', 'csushpinsa_lag3', 'csushpinsa_lag4', 'csushpinsa_lag5', 'csushpinsa_lag6', 'ccmls_cnt_full_btrm', 'clmls_cnt_full_btrm', 'fatax_cnt_full_btrm', 'ucdp_cnt_full_btrm_hve', 'ccmls_cnt_half_btrm', 'clmls_cnt_half_btrm', 'ucdp_cnt_hlf_btrm_hve', 'ucdp_improvementsbasementarea', 'fatax_nbr_bsmt_sq_ft', 'ccmls_nbr_area_bsmt', 'clmls_nbr_area_bsmt', 'fatax_nbr_grge_sq_ft', 'clmls_amt_list_prce', 'ccmls_list_prce','clmls_desc_pub_rmrk', 'ccmls_text_pub_lstg_cmnt', 'ccmls_desc_aplnc', 'ccmls_desc_eqpmnt_othr','final_zip', 'final_state', 'final_statecounty', 'geohash6', 'geohash5', 'geohash4', 'geohash3', 'geohash2', 'H8', 'H7', 'H6', 'H5', 'H4', 'H3', 'H2', 'quarter', 'ccmls_type_ppty_hve', 'clmls_ppty_type', 'fnm_type_ppty_hve', 'fre_type_ppty_hve', 'idmtax_type_ppty_hve', 'ucdp_type_ppty_hve', 'clmls_type_cond', 'ccmls_type_cond', 'ucdp_condition_rating', 'idmtax_type_bldng_cnsttn_qual_hve', 'fatax_type_bldng_cnsttn_qual_hve', 'ucdp_qualityconstr_hve', 'fatax_cd_absentee_ownr', 'idmtax_cd_ownr_ocpd', 'fatax_type_bldng_styl', 'idmtax_type_bldng_styl', 'ccmls_type_home_styl', 'clmls_type_bldng_styl_cl', 'fatax_type_grge', 'idmtax_type_grge', 'ccmls_type_grge',  'fatax_type_bsmt', 'idmtax_type_bsmt', 'ccmls_type_bsmt', 'clmls_type_bsmt', 'ccmls_poor_cond_sale', 'clmls_poor_cond_sale', 'ccmls_renovated_sale', 'clmls_renovated_sale', 'ucdp_renovated_sale', 'ccmls_ind_new_cntrctn', 'clmls_ind_new_cntrctn', 'ucdp_ind_new_cntrctn', 'ccmls_sub_cond_sale', 'clmls_sub_cond_sale', 'final_address', 'ntdtefnd_lag1', 'final_ntdtefnd', 'clmls_dttm_list_orig_cl','sale_age_lag1', 'sale_age_toanchr', 'ccmls_dttm_mls_evnt', 'sale_age_toanchr_raw', 'final_statecounty_raw', 'final_state_raw', 'log_trgt']]
X_train_s=X_train_s[['ccmls_cnt_bdrm', 'ccmls_cnt_btrm', 'ccmls_cnt_ppty_stry', 'ccmls_cnt_unit', 'ccmls_dt_blt_yr', 'ccmls_nbr_gla', 'clmls_cnt_bdrm', 'clmls_cnt_tot_btrm', 'clmls_cnt_bldng_stry_cl', 'clmls_cnt_unit', 'clmls_cnt_grge_car', 'clmls_dt_blt_yr', 'clmls_nbr_area_livng', 'clmls_cnt_prkng_spce_open','clmls_cnt_prkng_spce_tot', 'fre_cnt_unit', 'fatax_amt_tot_asesd_val', 'fatax_amt_asesd_imprmt_val', 'fatax_amt_asesd_lnd_val', 'fatax_cnt_bdrm', 'fatax_cnt_bldng_stry', 'fatax_cnt_prkng_spce', 'fatax_cnt_tot_btrm_hve', 'fatax_cnt_unit', 'fatax_dt_eff_blt_yr', 'fatax_nbr_livng_sq_ft', 'fatax_nbr_lot_size_sq_ft', 'fatax_amt_tot_mkt_val', 'fatax_dt_cnsttn_yr', 'fatax_dt_tax_yr', 'idmtax_amt_asesd_tot_val', 'idmtax_amt_asesd_imprmt_val', 'idmtax_amt_asesd_lnd_val', 'idmtax_cnt_bdrm', 'idmtax_cnt_bldng_stry_drvd', 'idmtax_cnt_grge_car', 'idmtax_cnt_tot_btrm_hve', 'idmtax_cnt_unit', 'idmtax_dt_cnsttn_yr', 'idmtax_nbr_lot_size_sq_ft', 'idmtax_nbr_unvrsl_bldng_sq_ft_drvd', 'idmtax_amt_tot_mkt_val', 'idmtax_dt_tax_yr', 'lat', 'long', 'ucdp_cnt_prkng_spce', 'ucdp_cnt_tot_btrm_hve', 'ucdp_cnt_bdrm', 'ucdp_cnt_unit_hve', 'ucdp_effectiveage', 'ucdp_nbr_bldng_sq_ft_hve', 'ucdp_num_grge_hve', 'ucdp_quality_of_construction', 'ucdp_site_sqft', 'ucdp_stories', 'ucdp_year_built', 'ucdp_salescompsubjactualage', 'mortgage_rate_lag1', 'mortgage_rate_lag2', 'mortgage_rate_lag3', 'mortgage_rate_lag4', 'mortgage_rate_lag5', 'mortgage_rate_lag6', 'avg_price_3m_bdrm_geohash6_orig','avg_price_6m_bdrm_geohash6_orig','avg_price_12m_bdrm_geohash6_orig', 'purprice_lag1', 'csushpinsa_lag1', 'csushpinsa_lag2', 'csushpinsa_lag3', 'csushpinsa_lag4', 'csushpinsa_lag5', 'csushpinsa_lag6', 'ccmls_cnt_full_btrm', 'clmls_cnt_full_btrm', 'fatax_cnt_full_btrm', 'ucdp_cnt_full_btrm_hve', 'ccmls_cnt_half_btrm', 'clmls_cnt_half_btrm', 'ucdp_cnt_hlf_btrm_hve', 'ucdp_improvementsbasementarea', 'fatax_nbr_bsmt_sq_ft', 'ccmls_nbr_area_bsmt', 'clmls_nbr_area_bsmt', 'fatax_nbr_grge_sq_ft', 'clmls_amt_list_prce', 'ccmls_list_prce','clmls_desc_pub_rmrk', 'ccmls_text_pub_lstg_cmnt', 'ccmls_desc_aplnc', 'ccmls_desc_eqpmnt_othr','final_zip', 'final_state', 'final_statecounty', 'geohash6', 'geohash5', 'geohash4', 'geohash3', 'geohash2', 'H8', 'H7', 'H6', 'H5', 'H4', 'H3', 'H2', 'quarter', 'ccmls_type_ppty_hve', 'clmls_ppty_type', 'fnm_type_ppty_hve', 'fre_type_ppty_hve', 'idmtax_type_ppty_hve', 'ucdp_type_ppty_hve', 'clmls_type_cond', 'ccmls_type_cond', 'ucdp_condition_rating', 'idmtax_type_bldng_cnsttn_qual_hve', 'fatax_type_bldng_cnsttn_qual_hve', 'ucdp_qualityconstr_hve', 'fatax_cd_absentee_ownr', 'idmtax_cd_ownr_ocpd', 'fatax_type_bldng_styl', 'idmtax_type_bldng_styl', 'ccmls_type_home_styl', 'clmls_type_bldng_styl_cl', 'fatax_type_grge', 'idmtax_type_grge', 'ccmls_type_grge', 'fatax_type_bsmt', 'idmtax_type_bsmt', 'ccmls_type_bsmt', 'clmls_type_bsmt', 'ccmls_poor_cond_sale', 'clmls_poor_cond_sale', 'ccmls_renovated_sale', 'clmls_renovated_sale', 'ucdp_renovated_sale', 'ccmls_ind_new_cntrctn', 'clmls_ind_new_cntrctn', 'ucdp_ind_new_cntrctn', 'ccmls_sub_cond_sale', 'clmls_sub_cond_sale', 'final_address', 'ntdtefnd_lag1', 'final_ntdtefnd', 'clmls_dttm_list_orig_cl','sale_age_lag1', 'sale_age_toanchr', 'ccmls_dttm_mls_evnt', 'sale_age_toanchr_raw', 'final_statecounty_raw', 'final_state_raw', 'log_trgt']]

In [13]:
trgt_mean = preprocessor.named_transformers_['std_trgt'].mean_[0]
trgt_s = preprocessor.named_transformers_['std_trgt'].scale_[0]
print(trgt_mean,trgt_s)

13.20246045067208 0.6203030147821884


In [14]:
x_cols_ple =preprocessor.transformers_[0][2]
x_cols_other = adj_col_list(X_train_s.copy().columns.values.tolist(),x_cols_ple,retain=False)
x_cols_other.remove('final_address')

In [15]:
###Define the data loaders - This is for training
class MLP_Dataset_noleak(Dataset):
    def __init__(self,X,trgt_col_name):#,state):
        super(MLP_Dataset_noleak,self).__init__()
        self.col_idx_dic = {}
        for col in X.columns:
             self.col_idx_dic[col] = X.columns.get_loc(col)
        ##Define Model columns
        self.trgt_col_name = trgt_col_name
        # self.preprocsr = preprocsr
        self.X_ple = np.array(X[x_cols_ple].copy().to_numpy().tolist(),dtype=float)
        self.X_other = X[x_cols_other].copy().to_numpy().astype(float)
        self.X_txt = np.array(X[['clmls_desc_pub_rmrk','ccmls_text_pub_lstg_cmnt','ccmls_desc_aplnc','ccmls_desc_eqpmnt_othr']].copy().to_numpy().tolist(),dtype=float)
    def __len__(self):
        return len(self.X_ple) 
    
    def __getitem__(self, idx):
        rows_ple = self.X_ple[idx]
        rows_other = self.X_other[idx]
        rows_txt = self.X_txt[idx]
        rows_procs = {**{k: (rows_ple[v] if k in x_cols_ple else rows_other[v-len(x_cols_ple)-4]) for k,v in self.col_idx_dic.items() } ,  
                      **{'CLS': -1},
                      **{'clmls_desc_pub_rmrk': rows_txt[0]},
                      **{'ccmls_text_pub_lstg_cmnt': rows_txt[1]},
                      **{'ccmls_desc_aplnc': rows_txt[2]},
                      **{'ccmls_desc_eqpmnt_othr': rows_txt[3]},
                      }
        rows_procs = {k: torch.squeeze(torch.tensor(np.array(v)).float(),dim=0) if k in x_cols_ple+['clmls_desc_pub_rmrk','ccmls_text_pub_lstg_cmnt','ccmls_desc_aplnc','ccmls_desc_eqpmnt_othr'] else torch.tensor(np.array(v)).unsqueeze(0).float() for k,v in rows_procs.items()}
        target=rows_procs[self.trgt_col_name]
        del rows_procs[self.trgt_col_name]
        return rows_procs, target

In [16]:
##Since data can be very large goal is to avoid creating copies of data.
##Instead of having a data loader class we will use a function to return what we need
def MLP_get_dataloader(dataset_fn,targtCol,df_train,df_test=None,split_train_frac=None,batchSize=500,numWorkers=0):
    
    test_loader = None
    val_loader =None
    
    if df_test is not None:
        dataset_test = dataset_fn(df_test,targtCol)
        test_loader = DataLoader(dataset_test,batch_size=batchSize,num_workers=numWorkers,shuffle=False)
    if split_train_frac is not None:
        train_index = pd.Index(np.random.choice(df_train.index, int(df_train.shape[0]*split_train_frac),replace=False))
        dataset_train = dataset_fn(df_train.iloc[train_index],targtCol)
        dataset_val = dataset_fn(df_train.drop(train_index),targtCol)
        val_loader= DataLoader(dataset_val,batch_size=batchSize,num_workers=numWorkers,shuffle=True)
    else:
        dataset_train = dataset_fn(df_train,targtCol)
    train_loader= DataLoader(dataset_train,batch_size=batchSize,num_workers=numWorkers,shuffle=True)
    return train_loader,val_loader,test_loader

In [17]:
X_train_s = X_train_s.drop(columns=['final_address'])
X_test_s = X_test_s.drop(columns=['final_address'])

In [18]:
##Instantiate the data loader objects
x_cols_ple =preprocessor.transformers_[0][2]
x_cols_other = adj_col_list(X_train_s.copy().columns.values.tolist(),x_cols_ple+['clmls_desc_pub_rmrk','ccmls_text_pub_lstg_cmnt','ccmls_desc_aplnc','ccmls_desc_eqpmnt_othr'],retain=False)
remove_cols = ['ntdtefnd_lag1','final_ntdtefnd','clmls_dttm_list_orig_cl','ccmls_dttm_mls_evnt','sale_age_toanchr_raw','final_statecounty_raw','final_state_raw']#'final_address',
for col in remove_cols:
    x_cols_other.remove(col)
    
    
##Instantiate the data loader objects
train_loader_MLP,val_loader_MLP,test_loader_MLP=MLP_get_dataloader(dataset_fn = MLP_Dataset_noleak,
                                                                   targtCol = 'log_trgt',
                                                                    df_train = X_train_s.drop(columns=['ntdtefnd_lag1','final_ntdtefnd','clmls_dttm_list_orig_cl','ccmls_dttm_mls_evnt','sale_age_toanchr_raw','final_statecounty_raw','final_state_raw']).copy(),
                                                                   df_test= X_test_s.drop(columns=['ntdtefnd_lag1','final_ntdtefnd','clmls_dttm_list_orig_cl','ccmls_dttm_mls_evnt','sale_age_toanchr_raw','final_statecounty_raw','final_state_raw']).copy(),
                                                                   split_train_frac = 0.9,
                                                                   batchSize = 700,
                                                                   numWorkers=6
                                                                   )

In [19]:
##Clear some memory 
del X_train_s
del X_test_s
gc.collect()

0

In [20]:
def emd_dim_cal(cat_size):    
    #For emedding size info https://ai.stackexchange.com/questions/28564/how-to-determine-the-embedding-size
    if cat_size <= 100:
        return int(min(50,cat_size/2))
    elif 100 < cat_size <= 1000:
        return int(min(500,cat_size/2))
    else:
        return int(75.6496 * np.log(cat_size + 176.623) - 41.4457)

In [21]:
input_dict

{'ccmls_cnt_bdrm': 45,
 'ccmls_cnt_btrm': 45,
 'ccmls_cnt_ppty_stry': 45,
 'ccmls_cnt_unit': 45,
 'ccmls_dt_blt_yr': 45,
 'ccmls_nbr_gla': 45,
 'clmls_cnt_bdrm': 45,
 'clmls_cnt_tot_btrm': 45,
 'clmls_cnt_bldng_stry_cl': 45,
 'clmls_cnt_unit': 45,
 'clmls_cnt_grge_car': 45,
 'clmls_dt_blt_yr': 45,
 'clmls_nbr_area_livng': 45,
 'clmls_cnt_prkng_spce_open': 45,
 'clmls_cnt_prkng_spce_tot': 45,
 'fre_cnt_unit': 45,
 'fatax_amt_tot_asesd_val': 45,
 'fatax_amt_asesd_imprmt_val': 45,
 'fatax_amt_asesd_lnd_val': 45,
 'fatax_cnt_bdrm': 45,
 'fatax_cnt_bldng_stry': 45,
 'fatax_cnt_prkng_spce': 45,
 'fatax_cnt_tot_btrm_hve': 45,
 'fatax_cnt_unit': 45,
 'fatax_dt_eff_blt_yr': 45,
 'fatax_nbr_livng_sq_ft': 45,
 'fatax_nbr_lot_size_sq_ft': 45,
 'fatax_amt_tot_mkt_val': 45,
 'fatax_dt_cnsttn_yr': 45,
 'fatax_dt_tax_yr': 45,
 'idmtax_amt_asesd_tot_val': 45,
 'idmtax_amt_asesd_imprmt_val': 45,
 'idmtax_amt_asesd_lnd_val': 45,
 'idmtax_cnt_bdrm': 45,
 'idmtax_cnt_bldng_stry_drvd': 45,
 'idmtax_cnt_grge

In [22]:
output_dict=dict()
for k,v in input_dict.items():
    output_dict[k]=emd_dim_cal(v)

In [23]:
output_dict

{'ccmls_cnt_bdrm': 22,
 'ccmls_cnt_btrm': 22,
 'ccmls_cnt_ppty_stry': 22,
 'ccmls_cnt_unit': 22,
 'ccmls_dt_blt_yr': 22,
 'ccmls_nbr_gla': 22,
 'clmls_cnt_bdrm': 22,
 'clmls_cnt_tot_btrm': 22,
 'clmls_cnt_bldng_stry_cl': 22,
 'clmls_cnt_unit': 22,
 'clmls_cnt_grge_car': 22,
 'clmls_dt_blt_yr': 22,
 'clmls_nbr_area_livng': 22,
 'clmls_cnt_prkng_spce_open': 22,
 'clmls_cnt_prkng_spce_tot': 22,
 'fre_cnt_unit': 22,
 'fatax_amt_tot_asesd_val': 22,
 'fatax_amt_asesd_imprmt_val': 22,
 'fatax_amt_asesd_lnd_val': 22,
 'fatax_cnt_bdrm': 22,
 'fatax_cnt_bldng_stry': 22,
 'fatax_cnt_prkng_spce': 22,
 'fatax_cnt_tot_btrm_hve': 22,
 'fatax_cnt_unit': 22,
 'fatax_dt_eff_blt_yr': 22,
 'fatax_nbr_livng_sq_ft': 22,
 'fatax_nbr_lot_size_sq_ft': 22,
 'fatax_amt_tot_mkt_val': 22,
 'fatax_dt_cnsttn_yr': 22,
 'fatax_dt_tax_yr': 22,
 'idmtax_amt_asesd_tot_val': 22,
 'idmtax_amt_asesd_imprmt_val': 22,
 'idmtax_amt_asesd_lnd_val': 22,
 'idmtax_cnt_bdrm': 22,
 'idmtax_cnt_bldng_stry_drvd': 22,
 'idmtax_cnt_grge

In [24]:
def getPositionEncoding(seq_len, d, n=10000):
    #n : User-defined scalar, set to 10,000 by the authors of Attention Is All You Need.
    P = np.zeros((seq_len, d))
    for k in range(seq_len):
        for i in np.arange(int(d/2)):
            denominator = np.power(n, 2*i/d)
            P[k, 2*i] = np.sin(k/denominator)
            P[k, 2*i+1] = np.cos(k/denominator)
    return P
 
P = getPositionEncoding(seq_len=3, d=6, n=10000)
print(P)


[[ 0.          1.          0.          1.          0.          1.        ]
 [ 0.84147098  0.54030231  0.04639922  0.99892298  0.00215443  0.99999768]
 [ 0.90929743 -0.41614684  0.0926985   0.99569422  0.00430886  0.99999072]]


In [25]:
def t2v(tau, f, out_features, w, b, w0, b0, arg=None):
    if arg:
        v1 = f(torch.matmul(tau, w) + b, arg)
    else:
        #print(w.shape, t1.shape, b.shape)
        v1 = f(torch.matmul(tau, w) + b)
    v2 = torch.matmul(tau, w0) + b0
    #print(v1.shape)
    return torch.cat([v1, v2], -1)

In [26]:
class CosineActivation(nn.Module):
    def __init__(self, in_features, out_features):
        super(CosineActivation, self).__init__()
        self.out_features = out_features
        self.w0 = nn.parameter.Parameter(torch.randn(in_features, 1))
        self.b0 = nn.parameter.Parameter(torch.randn(1))
        self.w = nn.parameter.Parameter(torch.randn(in_features, out_features-1))
        self.b = nn.parameter.Parameter(torch.randn(out_features-1))
        self.f = torch.cos

    def forward(self, tau):
        return t2v(tau, self.f, self.out_features, self.w, self.b, self.w0, self.b0)

In [27]:
class SineActivation(nn.Module):
    def __init__(self, in_features, out_features):
        super(SineActivation, self).__init__()
        self.out_features = out_features
        self.w0 = nn.parameter.Parameter(torch.randn(in_features, 1))
        self.b0 = nn.parameter.Parameter(torch.randn(1))
        self.w = nn.parameter.Parameter(torch.randn(in_features, out_features-1))
        self.b = nn.parameter.Parameter(torch.randn(out_features-1))
        self.f = torch.sin

    def forward(self, tau):
        return t2v(tau, self.f, self.out_features, self.w, self.b, self.w0, self.b0)

In [28]:
class custom_vsn(torch.nn.Module):
    def __init__(self,feat_num,in_dim,hidden_dim,out_dim,context_size=None, dropout=.1,norm=False):
        super(custom_vsn,self).__init__()
        self.feat_num = feat_num
        self.in_dim = in_dim
        self.hidden_dim =hidden_dim
        self.out_dim = out_dim
        self.dropout = dropout
        self.norm = norm
        self.context_size = context_size   
        self.softmax = torch.nn.Softmax(dim=1)    
        ##layers for custom vsn
        self.vsn_gate = GatedResidualNetwork(in_dim,hidden_dim,out_dim,dropout=.05,residual=True,context_size=context_size)
        if self.norm:
            self.vsn_GRN_norm = torch.nn.LayerNorm(out_dim, eps=1e-05, elementwise_affine=True)
            
    def forward(self, x_feats, context = None):
        ##VSN  
        x_vsn = torch.stack(x_feats,dim=1)

        if self.context_size  is not None:
                vsn_context = torch.tile(context.unsqueeze(1),(1,self.feat_num,1))
                grn_weight = self.vsn_gate(x_vsn,context=vsn_context)
        else:
                grn_weight = self.vsn_gate(x_vsn)

        grn_weight = self.softmax(grn_weight)
        x_vsn = torch.sum(x_vsn*grn_weight,1)

        if self.norm:
                x_vsn = self.vsn_GRN_norm(x_vsn)
        return x_vsn


In [29]:
class self_attention(nn.Module):
    def __init__(self, input_dim):
        super(self_attention, self).__init__()
        self.input_dim = input_dim
        self.query = nn.Linear(input_dim, input_dim)
        self.key = nn.Linear(input_dim, input_dim)
        self.value = nn.Linear(input_dim, input_dim)
        self.softmax = nn.Softmax(dim=2)
        
    def forward(self, x,get_attention=False):
        queries = self.query(x)
        keys = self.key(x)
        values = self.value(x)
        scores = torch.bmm(queries, keys.transpose(1, 2)) / (self.input_dim ** 0.5)
        attention = self.softmax(scores)
        weighted = torch.bmm(attention, values)
        if get_attention:
            return attention,weighted
        else:
            return weighted

In [30]:
class GEGLU(nn.Module):
    def forward(self, x):
        x, gates = x.chunk(2, dim = -1)
        return x * torch.nn.functional.gelu(x)

In [31]:
x_cols_embed_group = ['final_zip','final_statecounty', 'final_state',
               'geohash','geohash6','geohash5','geohash4','geohash3','geohash2','H8','H7','H6','H5','H4','H3','H2','quarter',
               'ccmls_type_ppty_hve','clmls_ppty_type','fnm_type_ppty_hve','fre_type_ppty_hve','idmtax_type_ppty_hve','ucdp_type_ppty_hve',
               'clmls_type_cond','ccmls_type_cond','ucdp_condition_rating',
               'idmtax_type_bldng_cnsttn_qual_hve','fatax_type_bldng_cnsttn_qual_hve', 'ucdp_qualityconstr_hve',
               'fatax_cd_absentee_ownr', 'idmtax_cd_ownr_ocpd',
               'fatax_type_bldng_styl', 'idmtax_type_bldng_styl', 'ccmls_type_home_styl', 'clmls_type_bldng_styl_cl',
               'fatax_type_grge','idmtax_type_grge', 'ccmls_type_grge', 'clmls_type_grge',
               'fatax_type_bsmt','idmtax_type_bsmt' , 'ccmls_type_bsmt','clmls_type_bsmt'] #
x_cols_embed_no_group = ['ccmls_poor_cond_sale', 'clmls_poor_cond_sale', 'ccmls_renovated_sale','clmls_renovated_sale', 'ucdp_renovated_sale',
               'ccmls_ind_new_cntrctn', 'clmls_ind_new_cntrctn', 'ucdp_ind_new_cntrctn', 'ccmls_sub_cond_sale','clmls_sub_cond_sale','clmls_desc_pub_rmrk','ccmls_text_pub_lstg_cmnt']

In [32]:
'''
var_grps = dict()
var_grps['beds'] = ['ucdp_cnt_bdrm', 'idmtax_cnt_bdrm', 'fatax_cnt_bdrm']#'ccmls_cnt_bdrm','clmls_cnt_bdrm',
var_grps['full_bath']=['fatax_cnt_full_btrm','ucdp_cnt_full_btrm_hve']#'ccmls_cnt_full_btrm','clmls_cnt_full_btrm',
var_grps['half_bath']=['ccmls_cnt_half_btrm','clmls_cnt_half_btrm','ucdp_cnt_hlf_btrm_hve']
var_grps['basement']=['ucdp_improvementsbasementarea', 'fatax_nbr_bsmt_sq_ft']#,'ccmls_nbr_area_bsmt','clmls_nbr_area_bsmt'
#var_grps['fire_place']=['ccmls_cnt_fp','clmls_cnt_fp','ucdp_cnt_fp_hve']
var_grps['baths'] = ['ucdp_cnt_tot_btrm_hve', 'idmtax_cnt_tot_btrm_hve', 'fatax_cnt_tot_btrm_hve'] #'ccmls_cnt_btrm','clmls_cnt_tot_btrm',
var_grps['sqft'] = ['ucdp_nbr_bldng_sq_ft_hve', 'idmtax_nbr_unvrsl_bldng_sq_ft_drvd',  'fatax_nbr_livng_sq_ft']#'clmls_nbr_area_livng','ccmls_nbr_gla',
var_grps['lotsize'] = ['fatax_nbr_lot_size_sq_ft', 'ucdp_site_sqft', 'idmtax_nbr_lot_size_sq_ft']
var_grps['yr_blt'] = ['ucdp_year_built', 'idmtax_dt_cnsttn_yr', 'fatax_dt_eff_blt_yr']#'fatax_dt_cnsttn_yr','ccmls_dt_blt_yr','clmls_dt_blt_yr',
var_grps['stories'] = [ 'idmtax_cnt_bldng_stry_drvd', 'ucdp_stories', 'fatax_cnt_bldng_stry'] #'ccmls_cnt_ppty_stry','clmls_cnt_bldng_stry_cl',
var_grps['units'] = ['ucdp_cnt_unit_hve', 'idmtax_cnt_unit',  'fatax_cnt_unit','fre_cnt_unit'] #'ccmls_cnt_unit','clmls_cnt_unit', 
var_grps['ppty_type'] = ['ucdp_type_ppty_hve', 'idmtax_type_ppty_hve',  'fnm_type_ppty_hve', 'fre_type_ppty_hve'] #'clmls_ppty_type', 'ccmls_type_ppty_hve',
var_grps['condition'] = ['ucdp_condition_rating'] #'ccmls_type_cond','clmls_type_cond',
var_grps['parking'] = ['idmtax_cnt_grge_car', 'ucdp_num_grge_hve','fatax_cnt_prkng_spce', 'ucdp_cnt_prkng_spce']#'clmls_cnt_prkng_spce_tot','clmls_cnt_grge_car', 'clmls_cnt_prkng_spce_open'
var_grps['garage_space']=['fatax_nbr_grge_sq_ft']
var_grps['quality'] = ['idmtax_type_bldng_cnsttn_qual_hve','fatax_type_bldng_cnsttn_qual_hve', 'ucdp_qualityconstr_hve']
var_grps['occupied']= ['idmtax_cd_ownr_ocpd','fatax_cd_absentee_ownr']
var_grps['tax_values'] = ['fatax_amt_tot_asesd_val',  'idmtax_amt_asesd_tot_val']#'fatax_amt_calc_tot_val','idmtax_amt_tot_mkt_val','fatax_amt_tot_mkt_val'
var_grps['land_values'] = ['fatax_amt_asesd_lnd_val','idmtax_amt_asesd_lnd_val']#'fatax_amt_calc_lnd_val','fatax_amt_mkt_lnd_val'
var_grps['imprmt_values'] = ['fatax_amt_asesd_imprmt_val','idmtax_amt_asesd_imprmt_val']#'fatax_amt_calc_imprmt_val','fatax_amt_mkt_imprmt_val'
var_grps['building_style']=['fatax_type_bldng_styl', 'idmtax_type_bldng_styl'] # 'ccmls_type_home_styl', 'clmls_type_bldng_styl_cl'
var_grps['garage_type']=['fatax_type_grge','idmtax_type_grge']#'ccmls_type_grge','clmls_type_grge'
var_grps['basement_type']=['fatax_type_bsmt','idmtax_type_bsmt']# ,'ccmls_type_bsmt','clmls_type_bsmt'
#var_grps['doc_year']=['fatax_dt_doc_yr', 'fatax_dt_doc_yr_prr']
#var_grps['tax_year']=['fatax_dt_tax_yr','idmtax_dt_tax_yr']
#var_grps['sale_type']=['fatax_cd_sale','fre_type_sale_trans_hve']
var_grps['list_price']=['clmls_amt_list_prce','ccmls_list_prce']
#var_grps['total_room']=['clmls_cnt_tot_rm','fatax_cnt_tot_rm','idmtax_cnt_tot_rm','ucdp_cnt_tot_rm_hve']
#var_grps['unemp_rate']=['unemployment_county_rate', 'unemployment_county_rate_mean12', 'unemployment_county_rate_mean6', 'unemployment_rate', 'unemployment_rate_mean12', 'unemployment_rate_mean6', 'unemployment_state_rate', 'unemployment_state_rate_mean12', 'unemployment_state_rate_mean6']
#var_grps['other'] =  ['purprice_lag1']
'''
var_grps = dict()
var_grps['beds'] = ['clmls_cnt_bdrm','ucdp_cnt_bdrm', 'idmtax_cnt_bdrm', 'ccmls_cnt_bdrm','fatax_cnt_bdrm']
var_grps['full_bath']=['ccmls_cnt_full_btrm','clmls_cnt_full_btrm','fatax_cnt_full_btrm','ucdp_cnt_full_btrm_hve']
var_grps['half_bath']=['ccmls_cnt_half_btrm','clmls_cnt_half_btrm','ucdp_cnt_hlf_btrm_hve']
var_grps['basement']=['ucdp_improvementsbasementarea', 'fatax_nbr_bsmt_sq_ft', 'ccmls_nbr_area_bsmt','clmls_nbr_area_bsmt']
#var_grps['fire_place']=['ccmls_cnt_fp','clmls_cnt_fp','ucdp_cnt_fp_hve']
var_grps['baths'] = ['clmls_cnt_tot_btrm', 'ucdp_cnt_tot_btrm_hve', 'idmtax_cnt_tot_btrm_hve', 'ccmls_cnt_btrm', 'fatax_cnt_tot_btrm_hve']
var_grps['sqft'] = ['clmls_nbr_area_livng', 'ucdp_nbr_bldng_sq_ft_hve', 'idmtax_nbr_unvrsl_bldng_sq_ft_drvd', 'ccmls_nbr_gla', 'fatax_nbr_livng_sq_ft']
var_grps['lotsize'] = ['fatax_nbr_lot_size_sq_ft', 'ucdp_site_sqft', 'idmtax_nbr_lot_size_sq_ft']
var_grps['yr_blt'] = ['ccmls_dt_blt_yr','ucdp_year_built', 'idmtax_dt_cnsttn_yr', 'fatax_dt_eff_blt_yr','clmls_dt_blt_yr','fatax_dt_cnsttn_yr']#
var_grps['stories'] = [ 'ccmls_cnt_ppty_stry','idmtax_cnt_bldng_stry_drvd', 'ucdp_stories','clmls_cnt_bldng_stry_cl', 'fatax_cnt_bldng_stry'] #
var_grps['units'] = ['ccmls_cnt_unit','clmls_cnt_unit', 'ucdp_cnt_unit_hve', 'idmtax_cnt_unit',  'fatax_cnt_unit','fre_cnt_unit'] #
var_grps['ppty_type'] = ['clmls_ppty_type', 'ucdp_type_ppty_hve', 'idmtax_type_ppty_hve', 'ccmls_type_ppty_hve', 'fnm_type_ppty_hve', 'fre_type_ppty_hve']
var_grps['condition'] = ['ccmls_type_cond','clmls_type_cond','ucdp_condition_rating'] #
var_grps['parking'] = ['idmtax_cnt_grge_car', 'clmls_cnt_grge_car', 'ucdp_num_grge_hve','fatax_cnt_prkng_spce', 'ucdp_cnt_prkng_spce','clmls_cnt_prkng_spce_tot']#,'clmls_cnt_prkng_spce_open'
var_grps['garage_space']=['fatax_nbr_grge_sq_ft']
var_grps['quality'] = ['idmtax_type_bldng_cnsttn_qual_hve','fatax_type_bldng_cnsttn_qual_hve', 'ucdp_qualityconstr_hve']
var_grps['occupied']= ['idmtax_cd_ownr_ocpd','fatax_cd_absentee_ownr']
var_grps['tax_values'] = ['fatax_amt_tot_asesd_val',  'idmtax_amt_asesd_tot_val']#'fatax_amt_calc_tot_val','idmtax_amt_tot_mkt_val','fatax_amt_tot_mkt_val'
var_grps['land_values'] = ['fatax_amt_asesd_lnd_val','idmtax_amt_asesd_lnd_val']#'fatax_amt_calc_lnd_val','fatax_amt_mkt_lnd_val'
var_grps['imprmt_values'] = ['fatax_amt_asesd_imprmt_val','idmtax_amt_asesd_imprmt_val']#'fatax_amt_calc_imprmt_val','fatax_amt_mkt_imprmt_val'
var_grps['building_style']=['fatax_type_bldng_styl', 'idmtax_type_bldng_styl', 'ccmls_type_home_styl', 'clmls_type_bldng_styl_cl']
var_grps['garage_type']=['fatax_type_grge','idmtax_type_grge', 'ccmls_type_grge']#
var_grps['basement_type']=['fatax_type_bsmt','idmtax_type_bsmt' , 'ccmls_type_bsmt','clmls_type_bsmt']
#var_grps['doc_year']=['fatax_dt_doc_yr', 'fatax_dt_doc_yr_prr']
#var_grps['tax_year']=['fatax_dt_tax_yr','idmtax_dt_tax_yr']
#var_grps['sale_type']=['fatax_cd_sale','fre_type_sale_trans_hve']
var_grps['list_price']=['clmls_amt_list_prce','ccmls_list_prce']
#var_grps['total_room']=['clmls_cnt_tot_rm','fatax_cnt_tot_rm','idmtax_cnt_tot_rm','ucdp_cnt_tot_rm_hve']
#var_grps['unemp_rate']=['unemployment_county_rate', 'unemployment_county_rate_mean12', 'unemployment_county_rate_mean6', 'unemployment_rate', 'unemployment_rate_mean12', 'unemployment_rate_mean6', 'unemployment_state_rate', 'unemployment_state_rate_mean12', 'unemployment_state_rate_mean6']
#var_grps['other'] =  ['purprice_lag1']

In [33]:
spatial_group = [ 'final_zip','final_statecounty','lat','long','final_state','H7','H6','H5','H4','H3','H2','geohash6','geohash5','geohash4','geohash3','geohash2']#'geohash',

In [34]:
txt_group_vsn =['clmls_desc_pub_rmrk','ccmls_text_pub_lstg_cmnt']
txt_group =['clmls_desc_pub_rmrk','ccmls_text_pub_lstg_cmnt','ccmls_desc_aplnc','ccmls_desc_eqpmnt_othr']

In [35]:
temporal_grps = dict()
temporal_grps['avp']=['avg_price_12m_bdrm_geohash6_orig','avg_price_6m_bdrm_geohash6_orig','avg_price_3m_bdrm_geohash6_orig']#'avg_price_12m',
temporal_grps['ushp']=[ 'csushpinsa_lag3', 'csushpinsa_lag2','csushpinsa_lag1']#,'csushpinsa','csushpinsa_lag6', 'csushpinsa_lag5', 'csushpinsa_lag4',
#var_grps['house_growth_avg']='csushpinsa_mean12', 'csushpinsa_mean6']
temporal_grps['mrtg_rate']=[ 'mortgage_rate_lag3', 'mortgage_rate_lag2', 'mortgage_rate_lag1']# 'mortgage_rate','mortgage_rate_lag6', 'mortgage_rate_lag5', 'mortgage_rate_lag4',
#var_grps_time['mortage_rate_avg']=['mortgage_rate_mean12', 'mortgage_rate_mean6']

In [36]:
class custom_self_attention(nn.Module):
    def __init__(self, input_dim,vdim=None, is_context=False, transformation=False):
        super(custom_self_attention, self).__init__()
        self.input_dim = input_dim
        self.vdim = vdim if vdim is not None else input_dim
        self.is_context=is_context
        if is_context:
            self.query = GatedResidualNetwork(input_dim,input_dim,input_dim,residual=True,context_size=45)#torch.nn.Identity() #nn.Linear(input_dim, input_dim) # #
            self.key = GatedResidualNetwork(input_dim,input_dim,input_dim,residual=True,context_size=45)#torch.nn.Identity() #nn.Linear(input_dim, input_dim)
        else:
            if transformation:
                self.query = nn.Linear(self.input_dim, self.input_dim,bias=False)#torch.nn.Identity()
                self.key = nn.Linear(self.input_dim, self.input_dim,bias=False)
            else:
                self.query =torch.nn.Identity()
                self.key=torch.nn.Identity()
        self.value = nn.Linear(self.vdim, self.vdim,bias=False)#torch.nn.Identity() #
        self.softmax = nn.Softmax(dim=2)
        
    def forward(self, x, xk=None, xv=None, attn_mask=None,get_attention=True,context_k=None,context_q=None):
        if self.is_context:
            context_k=torch.tile(context_k.unsqueeze(1),(1,xk.shape[1],1)) if xk is not None else torch.tile(context_k.unsqueeze(1),(1,x.shape[1],1))
            context_q=torch.tile(context_q.unsqueeze(1),(1,x.shape[1],1))
            queries = self.query(x,context_q)
            keys = self.key(xk,context_k) if xk is not None else self.key(x,context_k)
        else:
            queries = self.query(x)
            keys = self.key(xk) if xk is not None else self.key(x)
        values = xv if xv is not None else x #self.value(xv) if xv is not None else self.value(x)
        scores = torch.bmm(queries, keys.transpose(1, 2)) / (self.input_dim ** 0.5)
        
        ##Note issue when all keys are masked , so implmented hack solution suggested
        ## here https://github.com/pytorch/pytorch/issues/41508 to avoid nans in gradient
        
        if attn_mask is not None:
            # L, S = x.size(-2), xk.size(-2) if xk is not None else x.size(-2)
            # attn_bias = torch.zeros(L, S, dtype=x.dtype)
            #print(attn_mask)
            scores.masked_fill_(attn_mask, float("-inf"))
            #print(attn_bias)
            #attn_bias += attn_mask
            #scores += attn_bias
        #print(scores)    
            scores = self.softmax(scores)

            scores = scores.masked_fill(attn_mask,0)
        else:
            scores = self.softmax(scores)
        #print(scores)
        weighted = torch.bmm(scores, values)
        if get_attention:
            return weighted,scores
        else:
            return weighted

In [37]:
class SAINT(torch.nn.Module):
    def __init__(self,feat_num, feat_dim ,col_heads,cls_index = 0,return_cls=True,row_heads=None,kv=False,kv_feat_num=None,print_weight=False,mask=False,column_attn=False, is_context=False):
 
        super(SAINT,self).__init__()
        self.feat_num = feat_num
        self.feat_dim = feat_dim
        self.col_heads = col_heads
        self.cls_index = cls_index
        self.return_cls = return_cls
        self.kv=kv
        self.print_weight=print_weight
        self.mask=mask
        self.column_attn=column_attn
        self.is_context=is_context
        #self.kv_feat_num=kv_feat_num
        self.gelu = torch.nn.GELU()
        #SAINT - https://arxiv.org/pdf/2106.01342.pdf
        if is_context:
            self.selfatt_feat =custom_self_attention(feat_dim, is_context=True,transformation=True)
        else:
            self.selfatt_feat =custom_self_attention(feat_dim,transformation=True)
        self.selfatt_feat_norm = torch.nn.LayerNorm(feat_dim, eps=1e-05, elementwise_affine=True)
        
        self.feat_attn_fwd = torch.nn.Sequential(torch.nn.Linear(feat_dim,feat_dim),
                                                 self.gelu,
                                                 torch.nn.Linear(feat_dim,feat_dim),
                                                 self.gelu)
        self.selfatt_feat_norm2 = torch.nn.LayerNorm(feat_dim, eps=1e-05, elementwise_affine=True)
        
        ##Row attention 
        self.row_heads = row_heads if row_heads is not None else feat_num
        if kv:
            self.kv_transform=torch.nn.Linear(feat_dim*kv_feat_num,feat_dim*feat_num)
        #self.rowatt = torch.nn.MultiheadAttention(feat_dim*feat_num,self.row_heads,batch_first=True)
        self.rowatt = custom_self_attention(feat_dim*feat_num)
        self.rowatt_norm = torch.nn.LayerNorm(feat_dim, eps=1e-05, elementwise_affine=True)
        self.row_attn_fwd = torch.nn.Sequential(torch.nn.Linear(feat_dim,feat_dim),
                                                self.gelu,
                                                torch.nn.Linear(feat_dim,feat_dim),
                                                self.gelu)
        self.rowatt_norm2 = torch.nn.LayerNorm(feat_dim, eps=1e-05, elementwise_affine=True)        
        
    def forward(self,x,xk=None,xv=None,row_kv=None,mask_attn=None, context_k=None,context_q=None):
        if self.is_context:
            if xk is not None:
                x_feat_attn, col_weights =  self.selfatt_feat(x,xk,xv,context_k=context_k,context_q=context_q)
            else:
                x_feat_attn, col_weights =  self.selfatt_feat(x,x,x,context_k=context_k,context_q=context_q) ##out shape is [seq_len, batch,featDIM = projSize]
        else:
            if xk is not None:
                x_feat_attn, col_weights =  self.selfatt_feat(x,xk,xv)
            else:
                x_feat_attn, col_weights =  self.selfatt_feat(x,x,x)
        x_feat_attn = self.selfatt_feat_norm(x_feat_attn)
        x_feat_attn = x_feat_attn + x
        x_feat_attn_fwd = self.feat_attn_fwd(x_feat_attn)
        x_feat_attn_fwd = self.selfatt_feat_norm2(x_feat_attn_fwd)
        x_feat_attn = x_feat_attn_fwd + x_feat_attn
        if self.column_attn:
            if self.return_cls:
                x_feat_attn = x_feat_attn[:,self.cls_index,:]
            return x_feat_attn
        else:
            ##Now apply the row attention
            x_row = torch.flatten(x_feat_attn,start_dim=1).unsqueeze(0)
            if self.kv:
                row_kv_flat = self.kv_transform(torch.flatten(row_kv,start_dim=1).unsqueeze(0))
                if self.mask:
                    x_row_attn , row_weights = self.rowatt(x_row,row_kv_flat,row_kv_flat,attn_mask=mask_attn)
                else:
                    x_row_attn , row_weights = self.rowatt(x_row,row_kv_flat,row_kv_flat)
            else:
                if self.mask:
                    x_row_attn , row_weights = self.rowatt(x_row,x_row,x_row,attn_mask=mask_attn)
                else:
                    x_row_attn , row_weights = self.rowatt(x_row,x_row,x_row)
            if self.print_weight:
                print(weights)
            x_row_attn = x_row_attn.squeeze(0)
            x_row_attn = x_row_attn.unflatten(1,(self.feat_num ,self.feat_dim))
            x_row_attn = self.rowatt_norm(x_row_attn)
            x_row_attn = x_row_attn + x_feat_attn
            x_row_attn_fwd = self.row_attn_fwd(x_row_attn)
            x_row_attn_fwd = self.rowatt_norm2(x_row_attn_fwd)
            x_row_attn = x_row_attn_fwd + x_row_attn
            ##only take the cls token part
            if self.return_cls:
                x_row_attn = x_row_attn[:,self.cls_index,:]
            return x_row_attn

In [38]:
#from paper Delving into Deep Imbalanced Regression (ICML 2021, Long Oral) https://github.com/YyzHarry/imbalanced-regression/tree/main
def calibrate_mean_var(matrix, m1, v1, m2, v2, clip_min=0.1, clip_max=10):
    if torch.sum(v1) < 1e-10:
        return matrix
    if (v1 == 0.).any():
        valid = (v1 != 0.)
        factor = torch.clamp(v2[valid] / v1[valid], clip_min, clip_max)
        matrix[:, valid] = (matrix[:, valid] - m1[valid]) * torch.sqrt(factor) + m2[valid]
        return matrix

    factor = torch.clamp(v2 / v1, clip_min, clip_max)
    return (matrix - m1) * torch.sqrt(factor) + m2

In [39]:
#from paper Delving into Deep Imbalanced Regression (ICML 2021, Long Oral) https://github.com/YyzHarry/imbalanced-regression/tree/main
class FDS(nn.Module):

    def __init__(self, feature_dim, bucket_num=50, bucket_start=0, start_update=0, start_smooth=0,
                 kernel='gaussian', ks=5, sigma=2, momentum=0.9):
        super(FDS, self).__init__()
        self.feature_dim = feature_dim
        self.bucket_num = bucket_num
        self.bucket_start = bucket_start
        self.kernel_window = self._get_kernel_window(kernel, ks, sigma)
        self.half_ks = (ks - 1) // 2
        self.momentum = momentum
        self.start_update = start_update
        self.start_smooth = start_smooth

        self.register_buffer('epoch', torch.zeros(1).fill_(start_update))
        self.register_buffer('running_mean', torch.zeros(bucket_num - bucket_start, feature_dim))
        self.register_buffer('running_var', torch.ones(bucket_num - bucket_start, feature_dim))
        self.register_buffer('running_mean_last_epoch', torch.zeros(bucket_num - bucket_start, feature_dim))
        self.register_buffer('running_var_last_epoch', torch.ones(bucket_num - bucket_start, feature_dim))
        self.register_buffer('smoothed_mean_last_epoch', torch.zeros(bucket_num - bucket_start, feature_dim))
        self.register_buffer('smoothed_var_last_epoch', torch.ones(bucket_num - bucket_start, feature_dim))
        self.register_buffer('num_samples_tracked', torch.zeros(bucket_num - bucket_start))

    @staticmethod
    def _get_kernel_window(kernel, ks, sigma):
        assert kernel in ['gaussian', 'triang', 'laplace']
        half_ks = (ks - 1) // 2
        if kernel == 'gaussian':
            base_kernel = [0.] * half_ks + [1.] + [0.] * half_ks
            base_kernel = np.array(base_kernel, dtype=np.float32)
            kernel_window = gaussian_filter1d(base_kernel, sigma=sigma) / sum(gaussian_filter1d(base_kernel, sigma=sigma))
        elif kernel == 'triang':
            kernel_window = triang(ks) / sum(triang(ks))
        else:
            laplace = lambda x: np.exp(-abs(x) / sigma) / (2. * sigma)
            kernel_window = list(map(laplace, np.arange(-half_ks, half_ks + 1))) / sum(
                map(laplace, np.arange(-half_ks, half_ks + 1)))

        #logging.info(f'Using FDS: [{kernel.upper()}] ({ks}/{sigma})')
        return torch.tensor(kernel_window, dtype=torch.float32)#.cuda()

    def _get_bucket_idx(self, label):
        label = np.float32(label)
        _, bins_edges = np.histogram(a=np.array([], dtype=np.float32), bins=self.bucket_num, range=(-5, 5))
        if label == max(label):
            return self.bucket_num - 1
        else:
            return max(np.where(bins_edges > label)[0] - 1, self.bucket_start)

    def _update_last_epoch_stats(self):
        self.running_mean_last_epoch = self.running_mean
        self.running_var_last_epoch = self.running_var

        self.smoothed_mean_last_epoch = F.conv1d(
            input=F.pad(self.running_mean_last_epoch.unsqueeze(1).permute(2, 1, 0),
                        pad=(self.half_ks, self.half_ks), mode='reflect'),
            weight=self.kernel_window.view(1, 1, -1), padding=0
        ).permute(2, 1, 0).squeeze(1)
        self.smoothed_var_last_epoch = F.conv1d(
            input=F.pad(self.running_var_last_epoch.unsqueeze(1).permute(2, 1, 0),
                        pad=(self.half_ks, self.half_ks), mode='reflect'),
            weight=self.kernel_window.view(1, 1, -1), padding=0
        ).permute(2, 1, 0).squeeze(1)

    def reset(self):
        self.running_mean.zero_()
        self.running_var.fill_(1)
        self.running_mean_last_epoch.zero_()
        self.running_var_last_epoch.fill_(1)
        self.smoothed_mean_last_epoch.zero_()
        self.smoothed_var_last_epoch.fill_(1)
        self.num_samples_tracked.zero_()

    def update_last_epoch_stats(self, epoch):
        if epoch == self.epoch + 1:
            self.epoch += 1
            self._update_last_epoch_stats()
            #logging.info(f"Updated smoothed statistics of last epoch on Epoch [{epoch}]!")

    def update_running_stats(self, features, labels, epoch):
        if epoch < self.epoch:
            return

        assert self.feature_dim == features.size(1), "Input feature dimension is not aligned!"
        assert features.size(0) == labels.size(0), "Dimensions of features and labels are not aligned!"

        buckets = np.array([self._get_bucket_idx(label) for label in labels])
        for bucket in np.unique(buckets):
            curr_feats = features[torch.tensor((buckets == bucket).astype(np.uint8))]
            curr_num_sample = curr_feats.size(0)
            curr_mean = torch.mean(curr_feats, 0)
            curr_var = torch.var(curr_feats, 0, unbiased=True if curr_feats.size(0) != 1 else False)

            self.num_samples_tracked[bucket - self.bucket_start] += curr_num_sample
            factor = self.momentum if self.momentum is not None else \
                (1 - curr_num_sample / float(self.num_samples_tracked[bucket - self.bucket_start]))
            factor = 0 if epoch == self.start_update else factor
            self.running_mean[bucket - self.bucket_start] = \
                (1 - factor) * curr_mean + factor * self.running_mean[bucket - self.bucket_start]
            self.running_var[bucket - self.bucket_start] = \
                (1 - factor) * curr_var + factor * self.running_var[bucket - self.bucket_start]

        # make up for zero training samples buckets
        for bucket in range(self.bucket_start, self.bucket_num):
            if bucket not in np.unique(buckets):
                if bucket == self.bucket_start:
                    self.running_mean[0] = self.running_mean[1]
                    self.running_var[0] = self.running_var[1]
                elif bucket == self.bucket_num - 1:
                    self.running_mean[bucket - self.bucket_start] = self.running_mean[bucket - self.bucket_start - 1]
                    self.running_var[bucket - self.bucket_start] = self.running_var[bucket - self.bucket_start - 1]
                else:
                    self.running_mean[bucket - self.bucket_start] = (self.running_mean[bucket - self.bucket_start - 1] +
                                                                     self.running_mean[bucket - self.bucket_start + 1]) / 2.
                    self.running_var[bucket - self.bucket_start] = (self.running_var[bucket - self.bucket_start - 1] +
                                                                    self.running_var[bucket - self.bucket_start + 1]) / 2.
        #logging.info(f"Updated running statistics with Epoch [{epoch}] features!")

    def smooth(self, features, labels, epoch):
        if epoch < self.start_smooth:
            return features

        labels = labels.squeeze(1)
        buckets = np.array([self._get_bucket_idx(label) for label in labels])
        for bucket in np.unique(buckets):
            features[torch.tensor((buckets == bucket).astype(np.uint8))] = calibrate_mean_var(
                features[torch.tensor((buckets == bucket).astype(np.uint8))],
                self.running_mean_last_epoch[bucket - self.bucket_start],
                self.running_var_last_epoch[bucket - self.bucket_start],
                self.smoothed_mean_last_epoch[bucket - self.bucket_start],
                self.smoothed_var_last_epoch[bucket - self.bucket_start]
            )

        return features


In [40]:
# Define the neural network
class nn_vsn_saint(torch.nn.Module):
    def __init__(self,config, target_dim,projSize,var_input_sizes,vsn_grps,temporal_grps,use_cuda=False,start_smooth=0):
        
        super(nn_vsn_saint,self).__init__()
        
        ##For debugging
        self.print_switch = False
        ###Initialization values##
        self.target_dim = target_dim
        self.projSize = projSize
        self.var_input_sizes = var_input_sizes
        self.vsn_grps = vsn_grps
        self.temporal_grps = temporal_grps
        self.use_cuda = use_cuda
        self.min_embdd_size = 3
        ##Activations
        self.tanh = torch.nn.Tanh()
        self.celu = torch.nn.CELU(alpha=6)
        self.relu = torch.nn.ReLU()
        self.gelu = torch.nn.GELU()
        self.elu = torch.nn.ELU()
        self.sigmoid = torch.nn.Sigmoid()
        self.softmax = torch.nn.Softmax(dim=1)
        self.start_smooth=start_smooth
        ##Initialize the embedding based layers
        self.embed_layers = torch.nn.ModuleDict()
        #print('**Embedding Cols***')
        for k,v in var_input_sizes.items():
            if k in x_cols_embed_group:                
                self.embed_layers[k] =  torch.nn.Sequential(torch.nn.Embedding(v,output_dict[k]),
                                                            torch.nn.Linear(output_dict[k],projSize,bias=False)#GatedResidualNetwork(output_dict[k],projSize,projSize,dropout=.05)
                                                            )
            else:
                continue
        
        self.dummy_layers = torch.nn.ModuleDict()
        #print('**Dummy Cols***')
        for k,v in var_input_sizes.items():
            if k in x_cols_embed_no_group:                
                self.dummy_layers[k] =  torch.nn.Embedding(v,output_dict[k])
            else:
                continue
        
        ##Initialize  the linear projections
        self.proj_layers = torch.nn.ModuleDict()
        for k,v in var_input_sizes.items():
            if k  in x_cols_embed_group:
                self.proj_layers[k] = torch.nn.Identity()
            elif k  in x_cols_embed_no_group:
                continue
            elif k  in txt_group:
                continue
            #elif k  in txt_group_appr:
                #continue
            elif k == 'CLS':
                self.proj_layers[k] = torch.nn.Linear(v,projSize)
                self.proj_layers[k+'_avp'] = torch.nn.Linear(v,projSize,bias=False)
                self.proj_layers[k+'_ushp'] = torch.nn.Linear(v,projSize,bias=False)
                self.proj_layers[k+'_mrtg_rate'] = torch.nn.Linear(v,projSize,bias=False)
                self.proj_layers[k+'_spatial'] = torch.nn.Linear(v,projSize,bias=False)
                self.proj_layers[k+'_txt'] = torch.nn.Linear(v,projSize,bias=False)
                self.proj_layers[k+'_txt_appr'] = torch.nn.Linear(v,projSize,bias=False)
            elif k not in ['sale_age_toanchr','sale_age_lag1','purprice_lag1','H8','geohash']:
                self.proj_layers[k]  = torch.nn.Linear(v,projSize,bias=False) #GatedResidualNetwork(v,projSize,projSize,dropout=.05,residual=True)#
            else:
                continue
        
             
        self.temporal_layer_attn = torch.nn.ModuleDict()
        self.temporal_layer_norm = torch.nn.ModuleDict()
        for k,v in temporal_grps.items():
                self.temporal_layer_attn[k] = torch.nn.MultiheadAttention(projSize,15,batch_first=True)
                self.temporal_layer_norm[k] = torch.nn.LayerNorm(projSize, eps=1e-05, elementwise_affine=True)                  
            
        ##Initialize the VSN blocks
        self.vsn_layer = torch.nn.ModuleDict()
        for k,v in vsn_grps.items():
            if k == 'other':
                continue
            else:
                self.vsn_layer[k] = custom_vsn(len(v),projSize,projSize,projSize,dropout=config['vsn_dp'],norm=True, context_size=projSize)
                
        self.spatial_time_layer = torch.nn.ModuleDict()
        for k in spatial_group:
            self.spatial_time_layer[k] = GatedResidualNetwork(projSize,projSize,projSize,dropout=config['spatial_time_dp'],residual=True,context_size=45)#torch.nn.Identity() #
        
        self.txt_layer = torch.nn.ModuleDict()
        for k in txt_group:
            self.txt_layer[k] = torch.nn.Linear(384,projSize,bias=False)
            
        #self.txt_layer_appr = torch.nn.ModuleDict()
        #for k in txt_group_appr:
            #self.txt_layer_appr[k] = torch.nn.Linear(384,projSize,bias=False)
                                                    
        self.vsn_layer_txt = custom_vsn(2,projSize,projSize,projSize,dropout=config['vsn_dp'],norm=True)
        
        ##Projections and transformations related to time
        self.proj_time = SineActivation(1, projSize)#GatedResidualNetwork(var_input_sizes['sale_age_toanchr'],var_input_sizes['sale_age_toanchr'],var_input_sizes['sale_age_toanchr'],dropout=.0, residual=True)

        self.proj_time_lag1 = SineActivation(1, projSize)#GatedResidualNetwork(var_input_sizes['sale_age_lag1'],var_input_sizes['sale_age_lag1'],var_input_sizes['sale_age_lag1'],dropout=.0, residual=True)
        self.proj_dummy  = GatedResidualNetwork(10,projSize,projSize,dropout=config['dummy_dp']) 
        
        self.proj_prior_sale = GatedResidualNetwork(projSize,projSize,projSize,dropout=config['prior_sale_dp'],context_size=projSize) 
        
        self.saint_spatial=SAINT(len(spatial_group)+1, projSize ,3,cls_index=0,return_cls=True,row_heads=None) #config['heads_spatial']
        self.proj_spatial=GatedResidualNetwork(projSize,projSize,projSize,dropout=0.1,context_size=projSize)
        
        #self.saint_txt_appr=SAINT(2, projSize, 2,cls_index=0,return_cls=True,row_heads=None,column_attn=True)
        self.saint_txt=SAINT(4, projSize, 4,cls_index=0,return_cls=True,row_heads=None,is_context=True,column_attn=True)
        self.saint_txt_inter=SAINT(4, projSize, 4,cls_index=0,return_cls=True,is_context=True,row_heads=None)
        self.saint_feat=SAINT(27, projSize,15,cls_index=0,return_cls=True,row_heads=27,is_context=True)
        self.saint_feat_inter=SAINT(27, projSize,15,cls_index=0,return_cls=True,row_heads=27,is_context=True)
        self.saint_purprice_lag=SAINT(4, projSize,15,cls_index=0,return_cls=True,row_heads=15,kv=True,kv_feat_num=1)
        self.conv= nn.Conv1d(4,4,2, stride=1)
        self.avg_p_cv = nn.AvgPool1d(kernel_size=2, stride=1, padding=0)
        #self.bn=torch.nn.BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True)
        
        self.proj_feat  = GatedResidualNetwork(172,172,172,dropout=config['proj_dp'],residual=True) #GatedResidualNetwork(3*projSize,3*projSize,3*projSize,dropout=0,context_size=var_input_sizes['sale_age_toanchr'],residual=True)
        
        self.FDS = FDS(
                feature_dim=217, bucket_num=50, bucket_start=0,
                start_update=0, start_smooth=self.start_smooth, kernel='gaussian', ks=3, sigma=2, momentum=0.5
            )
        

        self.mlp_feat = torch.nn.Linear(217,target_dim,bias=True)#3*projSize
        #self.out_mlp_sigma = torch.nn.Linear(3*projSize+21,target_dim,bias=True)
        
    def forward(self,inputs,targets=None,epoch=None,fsd=False):
        
        targets=targets.unsqueeze(-1)
        ##Process the temporal blocks
        xt = self.proj_time(inputs['sale_age_toanchr'])#inputs['sale_age_toanchr']#
        inputs['sale_age_lag1'] =torch.nan_to_num(inputs['sale_age_lag1'])
        xt_lag1 =self.proj_time_lag1(inputs['sale_age_lag1'])#inputs['sale_age_lag1']#
        ##Process the embedding layers
        x_emdd = {}
        x_txt = []
        #x_txt_appr = []
        for k,v in inputs.items() :
            if k in self.embed_layers.keys():             
                x_emdd[k] = torch.squeeze(self.embed_layers[k](v.int()),dim=1)
            elif k in self.txt_layer.keys():
                x_txt.append(self.txt_layer[k](v))
            #elif k in self.txt_layer_appr.keys():
                #x_txt_appr.append(self.txt_layer_appr[k](v))
            else:
                continue
        x_dummy = {}
        for k,v in inputs.items() :
            if k in self.dummy_layers.keys():             
                x_dummy[k] = torch.squeeze(self.dummy_layers[k](v.int()),dim=1)
            else:
                continue

        ##Process the base projections
        x_proj = {}
        for k,v in inputs.items(): 
            if k in x_cols_embed_group:
                x_proj[k] = self.proj_layers[k](x_emdd[k])
            elif k in x_cols_embed_no_group:
                continue
            elif k in txt_group:
                continue
            #elif k in txt_group_appr:
                #continue
            elif k not in ['sale_age_toanchr','sale_age_lag1','purprice_lag1','H8','geohash']:
                x_proj[k] = self.proj_layers[k](v)
            else:
                continue

        x_temporal_out = {}
        for k,v in self.temporal_grps.items():
            x_proj['CLS_'+k] = self.proj_layers['CLS_'+k](inputs['CLS'])
            x_temporal_in = torch.stack([x_proj[i] for i in self.temporal_grps[k]]+[x_proj['CLS_'+k]] ,dim=1)
            x_positional =  getPositionEncoding(seq_len=len(v)+1, d=self.projSize, n=10000)
            x_temporal_in += torch.tensor(x_positional)
            temporal_attn,_ = self.temporal_layer_attn[k](x_temporal_in,x_temporal_in,x_temporal_in)
            x_temporal_out[k] = self.temporal_layer_norm[k](temporal_attn)[:,0,:]
        
        x_spatial_time = []
        for k in spatial_group:
            x_spatial_time.append(self.spatial_time_layer[k](x_proj[k],xt)) #
        
        ##Prior Sale Projection
        x_purprice_lag = self.proj_prior_sale(inputs['purprice_lag1'],xt_lag1)
        
        ##Dummies Projection
        x_dummy = torch.cat(list(x_dummy.values()),-1)
        x_dummy = self.proj_dummy(x_dummy)
        
        #process spatial attn
        x_proj['CLS_spatial'] = self.proj_layers['CLS_spatial'](inputs['CLS'])
        x_spatial = torch.stack([x_proj['CLS_spatial']]+x_spatial_time,dim=1)
        x_spatial_saint  = self.saint_spatial(x_spatial)
        
        txt_vsn_out=self.vsn_layer_txt(x_txt[:2])
        x_txt=[txt_vsn_out]+x_txt[2:]
        x_proj['CLS_txt'] = self.proj_layers['CLS_txt'](inputs['CLS'])
        x_txt = torch.stack([x_proj['CLS_txt']]+x_txt,dim=1)
        
        #x_proj['CLS_txt_appr'] = self.proj_layers['CLS_txt_appr'](inputs['CLS'])
        #x_txt_appr = torch.stack([x_proj['CLS_txt_appr']]+x_txt_appr,dim=1)
        
        
        
        ##Process the VSN Blocks
        x_vsn_out = {}
        for k,v in self.vsn_grps.items():
            if k =='other':
                continue
            else:    
                x_vsn_in = [x_proj[c] for c in v]
                x_vsn_out[k] = self.vsn_layer[k](x_vsn_in, context=x_spatial_saint)
        #x_list  = x_vsn_out['list_price']
        del x_vsn_out['list_price']
        
        if self.print_switch:
            print(x_vsn_out.keys())
            print(x_temporal_out.keys())
            self.print_switch = False

        #process feat attn
        x_feat  = torch.stack([x_proj['CLS'],x_dummy]+list(x_vsn_out.values())+list(x_temporal_out.values()),dim=1) #+list(x_temporal_out.values()),x_dummy
        #x_txt_appr_saint=self.saint_txt_appr(x_txt_appr)
        x_feat_saint= self.saint_feat_inter(x_feat,x_txt,x_txt,context_k=x_spatial_saint,context_q=x_spatial_saint)+self.saint_feat(x_feat,context_k=x_spatial_saint,context_q=x_spatial_saint)
        x_txt_saint  = self.saint_txt_inter(x_txt,x_feat,x_feat,context_k=x_spatial_saint,context_q=x_spatial_saint)+self.saint_txt(x_txt,context_k=x_spatial_saint,context_q=x_spatial_saint)
        x_purprice_lag_saint=self.saint_purprice_lag(torch.stack([x_purprice_lag]+list(x_temporal_out.values()),dim=1),row_kv=x_feat_saint)
        x_out = torch.stack([x_feat_saint,x_purprice_lag_saint,x_spatial_saint,x_txt_saint],dim=1)  #x_list #x_vsn_out['tax_values']
        x_out = torch.flatten(self.relu(self.avg_p_cv(self.conv(x_out))),start_dim=1)#.unsqueeze(1),.squeeze(1)
        x_out = self.proj_feat(x_out)
        ##Concat temporal and remaining feats
        x_out = torch.concat([xt,x_out],axis=1) #,xq
        
        ##output projection
        if epoch >= self.start_smooth and fsd:
                x_out = self.FDS.smooth(x_out, targets, epoch)#.permute(1, 0, 2)
        
        mew = self.mlp_feat(x_out)
        #sigma = torch.log(1 +torch.exp(self.out_mlp_sigma(x_out)))
        return mew

In [41]:
class loss_pc10_adaptive(nn.Module):
    def __init__(self):
        super(loss_pc10_adaptive, self).__init__()
        self.softmax=nn.Softmax()
    def forward(self, predictions, target):
        loss_value=0
        pred=torch.exp(predictions)
        label=torch.exp(target)
        #weight=self.softmax(torch.neg(target))
        length=list(predictions.size())[0]
        for i in range(length):
            ape=torch.abs(1-predictions[i]/target[i])
            if torch.abs(1-pred[i]/label[i])>0.1:
                    base_loss=ape
            else:
                base_loss=(40*(torch.square(torch.abs(predictions[i]-target[i])))+0.6)*ape
            loss_value+=base_loss
        return loss_value/length

In [42]:
class quantile_loss(nn.Module):
    def __init__(self, quantiles):
        super(quantile_loss, self).__init__()
        self.quantiles = quantiles
        
    # Loss function
    def get_one_quantile(self, y_pred, y_target, quantile):
        if quantile < 0 or quantile > 1:
            raise ValueError(
                'Illegal quantile value={}! Values should be between 0 and 1.'.format(quantile))

        pred_diff = y_target - y_pred
        loss = quantile * torch.max(pred_diff, torch.zeros_like(pred_diff)) + (1. - quantile) * torch.max(-pred_diff, torch.zeros_like(pred_diff))
        return loss.unsqueeze(-1)
    def forward(self, preds, targets):
        loss = []
        for i, quantile in enumerate(self.quantiles):
            loss.append(self.get_one_quantile(preds[..., i],targets, quantile))
        q_loss = torch.mean(torch.sum(torch.cat(loss, axis = -1), axis = -1))
        return q_loss

In [43]:
# ! pip install google-auth==2.22.0
# ! pip install tensorboard 

In [44]:
%%capture output
%%time

import torch
from torch.utils.tensorboard import SummaryWriter
from torch.nn import functional as F

# Configuration dictionary and model initialization
config = {
    'dummy_dp': 0.15,
    'vsn_dp': 0.05,
    'spatial_time_dp': 0.1,
    'prior_sale_dp': 0.03,
    'proj_dp': 0.1
}

model = nn_vsn_saint(config, 1, 45, input_dict, var_grps, temporal_grps)
model = model.float()

# Define loss and optimizer
loss_ap10 = loss_pc10_adaptive()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.005, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', threshold=1e-3, factor=.5, patience=0, min_lr=1e-7)

def rmse_loss(logits, labels):
    return torch.sqrt(F.mse_loss(logits, labels.reshape(-1, 1)))

def on_before_optimizer_step(optimizer, writer, global_step):
    if global_step % 25 == 0:  # Log every 25 steps
        for name, param in model.named_parameters():
            if param.grad is not None:
                writer.add_histogram(f"grad/{name}", param.grad, global_step)
                grad_norm = torch.linalg.vector_norm(param.grad).item()
                writer.add_scalar(f"grad/{name}_norm", grad_norm, global_step)

writer = SummaryWriter('runs/experiment_v6')

num_epochs = 50
global_step = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_pc10 = 0.0
    running_rmse = 0.0
    running_batch_cnt = 0

    for i, data in enumerate(train_loader_MLP):
        inputs_t, target_t = data
        optimizer.zero_grad()
        preds_t = model(inputs_t, target_t, epoch, fsd=True)

        # Loss calculations
        loss = loss_ap10(preds_t, target_t.float())
        rmse = rmse_loss(preds_t, target_t.float())
        pc10 = pc10_accm_torch(torch.exp(target_t), torch.exp(preds_t))

        running_loss += loss.item()
        running_rmse += rmse.item()  # Accumulate RMSE loss
        running_pc10 += pc10.item()
        running_batch_cnt += target_t.shape[0]

        # Backpropagation
        loss.backward()
        on_before_optimizer_step(optimizer, writer, global_step)  # Log gradients
        optimizer.step()
        global_step += 1  # Increment global step

    avg_loss = running_loss / len(train_loader_MLP)
    avg_rmse = running_rmse / len(train_loader_MLP)  # Average RMSE loss
    avg_pc10 = running_pc10 / running_batch_cnt

    # Log training metrics
    writer.add_scalar('Loss/Train', avg_loss, epoch)
    writer.add_scalar('RMSE/Train', avg_rmse, epoch)  # Log average RMSE loss
    writer.add_scalar('PC10/Train', avg_pc10, epoch)

    model.eval()
    running_loss_v = 0
    running_rmse_v = 0.0  # To accumulate RMSE loss for validation
    running_pc10_v = 0.0
    running_batch_cnt_v = 0

    with torch.no_grad():
        for i, data in enumerate(val_loader_MLP):
            inputs_v, target_v = data
            preds_v = model(inputs_v, target_v, epoch)
            loss_v = loss_ap10(preds_v, target_v.float())
            rmse_v = rmse_loss(preds_v, target_v.float())
            vpc10 = pc10_accm_torch(torch.exp(target_v), torch.exp(preds_v))

            running_loss_v += loss_v.item()
            running_rmse_v += rmse_v.item()  # Accumulate RMSE loss
            running_pc10_v += vpc10.item()
            running_batch_cnt_v += target_v.shape[0]

        avg_vloss = running_loss_v / len(val_loader_MLP)
        avg_vrmse = running_rmse_v / len(val_loader_MLP)  # Average RMSE loss for validation
        avg_vpc10 = running_pc10_v / running_batch_cnt_v

        # Log validation metrics
        writer.add_scalar('Loss/Validation', avg_vloss, epoch)
        writer.add_scalar('RMSE/Validation', avg_vrmse, epoch)  # Log average RMSE loss for validation
        writer.add_scalar('PC10/Validation', avg_vpc10, epoch)

    scheduler.step(avg_vpc10)
    print(f'Epoch {epoch}, Loss Train {avg_loss}, RMSE Train {avg_rmse}, PC10 Train {avg_pc10}, Loss Val {avg_vloss}, RMSE Val {avg_vrmse}, PC10 Val {avg_vpc10}')

# Close the writer
writer.close()


In [45]:
# %%capture output
# %%time

# import torch
# from torch.utils.tensorboard import SummaryWriter

# # Configuration dictionary and model initialization
# config = {
#     'dummy_dp': 0.15,
#     'vsn_dp': 0.05,
#     'spatial_time_dp': 0.1,
#     'prior_sale_dp': 0.03,
#     'proj_dp': 0.1
# }

# model = nn_vsn_saint(config, 1, 45, input_dict, var_grps, temporal_grps)
# model = model.float()

# # Define loss and optimizer
# loss_ap10 = loss_pc10_adaptive()
# optimizer = torch.optim.AdamW(model.parameters(), lr=0.005, weight_decay=0.01)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', threshold=1e-3, factor=.5, patience=0, min_lr=1e-7)

# # TensorBoard writer
# writer = SummaryWriter('runs/experiment_v2'1

# # Training loop
# num_epochs = 50
# for epoch in range(num_epochs):
#     model.train()
#     running_loss = 0
#     running_pc10 = 0.0
#     running_batch_cnt = 0.0

#     for i, data in enumerate(train_loader_MLP):
#         inputs_t, target_t = data
#         optimizer.zero_grad()
#         preds_t = model(inputs_t, target_t, epoch, fsd=True)
#         loss = loss_ap10(preds_t, target_t.float())
#         pc10 = pc10_accm_torch(torch.exp(target_t), torch.exp(preds_t))

#         running_loss += loss.item()
#         running_pc10 += pc10.item()
#         running_batch_cnt += target_t.shape[0]

#         loss.backward()
#         optimizer.step()

#     avg_loss = running_loss / (i + 1)
#     avg_pc10 = running_pc10 / running_batch_cnt

#     # Log training metrics
#     writer.add_scalar('Loss/Train', avg_loss, epoch)
#     writer.add_scalar('PC10/Train', avg_pc10, epoch)

#     model.eval()
#     running_loss_v = 0
#     running_pc10_v = 0.0
#     running_batch_cnt_v = 0.0

#     with torch.no_grad():
#         for i, data in enumerate(val_loader_MLP):
#             inputs_v, target_v = data
#             preds_v = model(inputs_v, target_v, epoch)
#             loss_v = loss_ap10(preds_v, target_v.float())
#             vpc10 = pc10_accm_torch(torch.exp(target_v), torch.exp(preds_v))

#             running_loss_v += loss_v.item()
#             running_pc10_v += vpc10.item()
#             running_batch_cnt_v += target_v.shape[0]

#         avg_vloss = running_loss_v / (i + 1)
#         avg_vpc10 = running_pc10_v / running_batch_cnt_v

#         # Log validation metrics
#         writer.add_scalar('Loss/Validation', avg_vloss, epoch)
#         writer.add_scalar('PC10/Validation', avg_vpc10, epoch)

#     scheduler.step(avg_vpc10)
#     print(f'Epoch {epoch}, Loss Train {avg_loss}, Loss Val {avg_vloss}, PC10 Train {avg_pc10}, PC10 Val {avg_vpc10}')

# # Close the writer
# writer.close()


In [46]:
output.show()

Epoch 0, Loss Train 3.4489622547461822, RMSE Train 1.1418155016483846, PC10 Train 0.08644138398972787, Loss Val 1.446762735779221, RMSE Val 1.0297294549039893, PC10 Val 0.09770425910180104
Epoch 1, Loss Train 2.369164626519601, RMSE Train 1.0617725471476536, PC10 Train 0.09143573689250761, Loss Val 1.0813624633325112, RMSE Val 0.9933193970370937, PC10 Val 0.10218752415552292
Epoch 2, Loss Train 1.8049958197920173, RMSE Train 1.0334603534088478, PC10 Train 0.09403812542138511, Loss Val 1.3358792614292454, RMSE Val 1.0092178357614052, PC10 Val 0.10110535672876246
Epoch 3, Loss Train 1.6540904378031824, RMSE Train 1.025036679373847, PC10 Train 0.09610371763655025, Loss Val 1.3124680937947453, RMSE Val 0.9969701718639683, PC10 Val 0.10137589858545258
Epoch 4, Loss Train 1.5545042907153521, RMSE Train 1.00428631320014, PC10 Train 0.09675216758351476, Loss Val 0.98933144517847, RMSE Val 0.9961666577571148, PC10 Val 0.0992888614052717
Epoch 5, Loss Train 1.0418910099579408, RMSE Train 1.00055

indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  ../aten/src/ATen/native/IndexingUtils.h:28.)
indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  ../aten/src/ATen/native/IndexingUtils.h:28.)
indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  ../aten/src/ATen/native/IndexingUtils.h:28.)


In [47]:
# %%capture output
# %%time
# #torch.autograd.set_detect_anomaly(True)
# config={'dummy_dp': 0.15, 'vsn_dp': 0.05, 'spatial_time_dp': 0.1, 'prior_sale_dp': 0.03, 'proj_dp': 0.1}
# ##Instantiate the model object
# model = nn_vsn_saint(config,1,45,input_dict,var_grps,temporal_grps)
# model= model.float()
# #model_weights = torch.load(f"/home/jovyan/geohash_index_output/Tab_transformer_customhve/CA/tabtransformer_{state[0]}_ALL_6.5_ple{ple_size}_{time}_17.pth",map_location=torch.device('cpu'))
# #model.load_state_dict(model_weights['model_state_dict'])
# ##Define the optimization plan

# #loss_mse = torch.nn.MSELoss()
# loss_ap10 = loss_pc10_adaptive()

# #loss_gnll = torch.nn.GaussianNLLLoss()
# l1_lambda = 0.05
# optimizer  = torch.optim.AdamW(model.parameters(), lr=0.005,weight_decay=0.01)#torch.optim.RAdam(model.parameters(), lr=0.005)#
# ##Define  Learning Rate Schedueler
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', threshold =1e-3,factor=.5, patience =0, min_lr= 1e-7)

# def get_lr(optimizer):
#     for p in optimizer.param_groups:
#         return p["lr"]
# ##Eexecute the training loop
# ##Eexecute the training loop
# num_epochs = 50

# ##Early stopping counter
# estopCntr = 0
# estopCntrLmt = 7
# estopBestVal = 0
# scoreBest = 0 
# current_vpc10=0
# for epoch in range(num_epochs):
#     #start_time = time.time()
#     model.train()
#     running_loss = 0 
#     running_pc10 = 0.0
#     running_batch_cnt=0.0
#     for i, data in enumerate(train_loader_MLP):
#     #for i, data in enumerate(train_loader_MLP):
#         optimizer.zero_grad()
#         inputs_t,target_t = data
#         preds_t= model(inputs_t,target_t,epoch,fsd=True)
#         loss=loss_ap10(preds_t*trgt_s+trgt_mean,target_t.float()*trgt_s+trgt_mean)     
#         pc10=pc10_accm_torch(torch.exp(target_t*trgt_s+trgt_mean),torch.exp(preds_t*trgt_s+trgt_mean))
#         running_loss += loss.item()
#         running_pc10 += pc10.item()
#         running_batch_cnt +=  target_t.shape[0]             
#         loss.backward()
#         optimizer.step()
#         gc.collect()
#     avg_loss = running_loss / (i + 1)
#     avg_pc10 = running_pc10 / running_batch_cnt
#     ##Model Evaluations on Validation and Test for current Epocj
#     model.eval()
#     with torch.no_grad():
#         ##Performance Stats on Validation Data
#         running_loss_v = 0 
#         running_pc10_v = 0.0
#         running_pc10plus_v = 0.0
#         running_batch_cnt_v = 0.0
#         for i, data in enumerate(val_loader_MLP):
#             inputs_v,target_v = data
#             preds_v = model(inputs_v,target_v,epoch)
#             loss_v =loss_ap10(preds_v*trgt_s+trgt_mean,target_v.float()*trgt_s+trgt_mean)
#             vpc10=pc10_accm_torch(torch.exp(target_v*trgt_s+trgt_mean),torch.exp(preds_v*trgt_s+trgt_mean))
#             vpc10plus = pc10plus_accm_torch(torch.exp(target_v*trgt_s+trgt_mean),torch.exp(preds_v*trgt_s+trgt_mean))
#             running_loss_v += loss_v.item()
#             running_pc10_v += vpc10.item()
#             running_pc10plus_v += vpc10plus.item()
#             running_batch_cnt_v +=  target_v.shape[0]
#         avg_vloss = running_loss_v / (i + 1)
#         avg_vpc10 = running_pc10_v / running_batch_cnt_v
#         avg_vpc10plus = running_pc10plus_v / running_batch_cnt_v
#         ##Performance Stats on Testing Data
#         running_loss_test = 0 
#         running_pc10_test = 0.0
#         running_pc10plus_test = 0.0
#         running_batch_cnt_test = 0.0
#         for i, data in enumerate(test_loader_MLP):
#             inputs_test,target_test = data
#             preds_test = model(inputs_test,target_test,epoch)
#             loss_test =loss_ap10(preds_test*trgt_s+trgt_mean,target_test.float()*trgt_s+trgt_mean)
#             testpc10=pc10_accm_torch(torch.exp(target_test*trgt_s+trgt_mean),torch.exp(preds_test*trgt_s+trgt_mean))
#             testpc10plus = pc10plus_accm_torch(torch.exp(target_test*trgt_s+trgt_mean),torch.exp(preds_test*trgt_s+trgt_mean))
#             running_loss_test += loss_test.item()
#             running_pc10_test += testpc10.item()
#             running_pc10plus_test += testpc10plus.item()
#             running_batch_cnt_test +=  target_test.shape[0]
#         avg_testloss = running_loss_test / (i + 1)
#         avg_testpc10 = running_pc10_test/ running_batch_cnt_test
#         avg_testpc10plus = running_pc10plus_test / running_batch_cnt_test
#     #old_vpc10=current_vpc10
#     # current_vpc10=avg_vpc10
#     ##Apply scheduelers and early stopping
#     scheduler.step(avg_vpc10)
#     print('Epoch {} , LOSS Train {} ,LOSS val {}, LOSS test {}, pc10_train {} ,pc10_val {}, pc10_test {},  pc10plus_test {} ,LR Next Epoch {} '.format(epoch,avg_loss,avg_vloss, avg_testloss, avg_pc10 ,avg_vpc10 , avg_testpc10, avg_testpc10plus, get_lr(scheduler.optimizer)))   
#     if avg_vpc10 > estopBestVal:
#         best_mdl = copy.deepcopy(model)
#         best_epoch = epoch
#         estopBestVal = avg_vpc10
#         estopCntr = 0
#     else:
#         estopCntr += 1
#         print('Estop Cntr Increased to ' , estopCntr)
#     gc.collect()
#     torch.cuda.empty_cache()
#     if (get_lr(scheduler.optimizer)==1e-7) or (estopCntr >= estopCntrLmt) :
#         print('*****Early Stopping Limit Reached******')
#         break

In [50]:
# output.show()

Epoch 0 , LOSS Train 0.017048900927397404 ,LOSS val 0.010223837544185084, LOSS test 0.013514482774413548, pc10_train 0.4365957666095515 ,pc10_val 0.5831336476772049, pc10_test 0.5239772727272727,  pc10plus_test 0.27375 ,LR Next Epoch 0.005 
Epoch 1 , LOSS Train 0.01022674687564731 ,LOSS val 0.010750500535642778, LOSS test 0.013700132114956012, pc10_train 0.577442530586654 ,pc10_val 0.5421658808069877, pc10_test 0.4709090909090909,  pc10plus_test 0.12056818181818182 ,LR Next Epoch 0.0025 
Estop Cntr Increased to  1
Epoch 2 , LOSS Train 0.009306732505034129 ,LOSS val 0.009338595375821396, LOSS test 0.011356300244537683, pc10_train 0.6210303912600972 ,pc10_val 0.6274638633377135, pc10_test 0.5939772727272727,  pc10plus_test 0.2109090909090909 ,LR Next Epoch 0.0025 
Epoch 3 , LOSS Train 0.008885973974972873 ,LOSS val 0.009471622346019422, LOSS test 0.011481586198967237, pc10_train 0.6369668002215895 ,pc10_val 0.6229033006106516, pc10_test 0.5877272727272728,  pc10plus_test 0.18738636363636

indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  ../aten/src/ATen/native/IndexingUtils.h:28.)
indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  ../aten/src/ATen/native/IndexingUtils.h:28.)


In [53]:
# Save the best model ##
# path = f"/home/jovyan/geohash_index_output/Tab_transformer_customhve/CA/tabtransformer_{state[0]}_ALL_7.6_ple{ple_size}_{time}_{best_epoch}.pth"
path = f"/home/jovyan/geohash_index_output/tab_transformer_customhve/tabtransformer_{state[0]}_ALL_7.6_ple{ple_size}_{time}_{best_epoch}.pth"

try:
    state_dict = best_mdl.module.state_dict()
    
except AttributeError:
    state_dict = best_mdl.state_dict()
torch.save({'model_state_dict': state_dict}, path)


In [54]:
##Get Performance Summary on Test Set##
config={'dummy_dp': 0.15, 'vsn_dp': 0.03, 'spatial_time_dp': 0.1, 'prior_sale_dp': 0.03, 'proj_dp': 0.1}
best_mdl = nn_vsn_saint(config,1,45,input_dict,var_grps,temporal_grps)
best_mdl= best_mdl.float()
model_weights = torch.load(f"/home/jovyan/geohash_index_output/tab_transformer_customhve/tabtransformer_{state[0]}_ALL_7.6_ple{ple_size}_{time}_{best_epoch}.pth",map_location=torch.device('cpu'))
best_mdl.load_state_dict(model_weights['model_state_dict'])
best_mdl.eval() #Best Model object
predicts_test=[]
with torch.no_grad():
    for i,data in enumerate(tqdm(test_loader_MLP)):
        inputs_x,target_x = data
        output_mew_pdp = best_mdl(inputs_x,target_x,epoch)
        predicts_test.append(output_mew_pdp.flatten())
    predicts_test=torch.cat(predicts_test)
    predicts_test.numpy()
bluesky_data_test['y_pred_t'] = predicts_test
bluesky_data_test['y_pred_t_dollar'] = np.exp(bluesky_data_test['y_pred_t'] * trgt_s + trgt_mean)

100%|██████████| 13/13 [00:15<00:00,  1.16s/it]


In [61]:
bluesky_data_test['lp_flag']=np.where((bluesky_data_test.clmls_amt_list_prce.isnull() & bluesky_data_test.ccmls_list_prce.isnull()) ,0, 1)
display(bluesky_data_test.groupby(['lp_flag']).apply(lambda d : df_metric_wArg(d,'final_purprice','y_pred_t_dollar')))

,count,pc10,pc10+,median_bias,mape
lp_flag,,,,,
0,2424.0,0.396,0.267,-0.022,0.376
1,6376.0,0.689,0.199,0.013,0.106


In [66]:
hve_prod = pd.read_pickle(f'{datapath}/hve_prod/hve_prod_{time}_statelevel.pkl')
hve_day0_cnty = hve_prod[(hve_prod.state==state[0])& (hve_prod.dayflag==0)]
df_metric_wArg(bluesky_data_test[bluesky_data_test.final_address.isin(hve_day0_cnty.address.values.tolist())],'final_purprice','y_pred_t_dollar').to_frame().T

,count,pc10,pc10+,median_bias,mape
0,6123.0,0.622,0.222,0.01,0.165


In [67]:
hve_prod = pd.read_pickle(f'{datapath}/hve_prod/hve_prod_{time}_statelevel.pkl')
hve_day0_cnty = hve_prod[(hve_prod.statecounty=="WA063")& (hve_prod.dayflag==0)]
df_metric_wArg(bluesky_data_test[bluesky_data_test.final_address.isin(hve_day0_cnty.address.values.tolist())],'final_purprice','y_pred_t_dollar').to_frame().T

,count,pc10,pc10+,median_bias,mape
0,452.0,0.586,0.19,-0.009,0.203


In [68]:
bluesky_data_test['lp_flag']=np.where((bluesky_data_test.clmls_amt_list_prce.isnull() & bluesky_data_test.ccmls_list_prce.isnull()) ,0, 1)
display(bluesky_data_test.groupby(['lp_flag']).apply(lambda d : df_metric_wArg(d,'final_purprice','y_pred_t_dollar')))

,count,pc10,pc10+,median_bias,mape
lp_flag,,,,,
0,2424.0,0.396,0.267,-0.022,0.376
1,6376.0,0.689,0.199,0.013,0.106


In [70]:
bluesky_data_test.to_pickle(f'{datapath}/with_pred/{state[0]}_{time}_ple{ple_size}_bluesky_data_test_with_pred_state.pkl')